In [1]:
%pylab inline
import pandas as pd
import seaborn as sns
import json
sns.set_context('paper', font_scale=2)
sns.set_style('white')
np.random.seed(42)

Populating the interactive namespace from numpy and matplotlib


/home/cmb-panasas2/skchoudh/software_frozen/anaconda27/envs/riboraptor/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/cmb-panasas2/skchoudh/software_frozen/anaconda27/envs/riboraptor/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/cmb-panasas2/skchoudh/software_frozen/anaconda27/envs/riboraptor/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [16]:
df = pd.read_csv('data/wdbc.data', header=None)
df = df.drop(columns=[0])
output = np.array(df.loc[:, 1])
df  = df.drop(columns=[1])


In [17]:
df.shape

(569, 30)

In [18]:
df_matrix = df.as_matrix()

/home/cmb-panasas2/skchoudh/software_frozen/anaconda27/envs/riboraptor/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [19]:
np.sum(df_matrix)

1056474.4596355997

# Normalize

In [20]:
df_normalized = df_matrix - df_matrix.mean()
df_normalized = df_normalized/np.sqrt((df_normalized * df_normalized).sum(axis=1))[:, np.newaxis]

In [21]:
df_normalized

array([[-0.01998381, -0.02344792,  0.02772619, ..., -0.02805213,
        -0.0279635 , -0.02811882],
       [-0.01797171, -0.01918952,  0.03088423, ..., -0.02683737,
        -0.02679866, -0.02687955],
       [-0.0208045 , -0.02003544,  0.03357715, ..., -0.03039166,
        -0.03033334, -0.03046828],
       ...,
       [-0.03326271, -0.02483149,  0.03408422, ..., -0.04535005,
        -0.04529129, -0.04539676],
       [-0.01915379, -0.01510415,  0.03627945, ..., -0.02858672,
        -0.02852006, -0.02865213],
       [-0.141273  , -0.09747973, -0.03646145, ..., -0.16152542,
        -0.16077614, -0.16134172]])

# Problem (a), (b)

In all of our experiments we will perform 100 random partitions of the data for
train/test and then report the final result as the average over these tria

\begin{align*}
(\hat{w}, \hat{b}) &= \text{arg} \min_{w,b} f(\vec{w}, b) := \frac{1}{N} \sum_{i=1}^N -y_i(\vec{w}^T\vec{x}_i + b) + \log(1+ \exp{(\vec{w}^T\vec{x}_i + b)} + \frac{\lambda^2}{2N} ||\vec{w}||_{l_2}^2
\end{align*}

Consider the derivatives:

\begin{align}
\frac{\partial f(\vec{w}, b)}{\partial b} &= \frac{1}{N} \sum_{i=1}^N -y_i + \frac{\exp{(\vec{w}^T\vec{x}_i + b)}}{1+ \exp{(\vec{w}^T\vec{x}_i + b)}}\\
&= \frac{1}{N} \sum_{i=1}^N \frac{1}{1+ \exp{(-(\vec{w}^T\vec{x}_i + b))}}-y_i
\end{align}

For $j \geq 1$ and learning rate $\mu$:

\begin{align}
\frac{\partial f(\vec{w}, b)}{\partial w_j} &= \frac{1}{N} \sum_{i=1}^N -y_i(x_j) + \frac{\exp{(\vec{w}^T\vec{x}_i + b)}}{1+ \exp{(\vec{w}^T\vec{x}_i + b)}}x_j + \frac{\lambda}{N} w_j\\ 
&=  \sum_{i=1}^N \big(\frac{1}{1+ \exp{(-(\vec{w}^T\vec{x}_i + b))}}-y_i\big)x_j + \frac{\lambda}{N}w_j
\end{align}

Gradient descent:

\begin{align*}
b^{(t+1)} &= b^{(t)} - \frac{\mu}{N} \sum_{i=1}^N \big(\frac{1}{1+ \exp{(-(\vec{w}^T\vec{x}_i + b))}}-y_i\big)\\
w^{(t+1)}_j &= w^{(t)}_j - \frac{\mu}{N} \sum_{i=1}^N \big(\frac{1}{1+ \exp{(-(\vec{w}^T\vec{x}_i + b))}}-y_i\big)x_{ij} + \frac{\lambda}{N}w_j^{(t)}\\
\end{align*}

where $x_{ij}$ refers to the $j^{th}$ feature of the $i^{th}$ sample. In our case.
$1 \leq j \leq 30$ and $i$ ranges over the number of samples.


This looks super confusing, let's resort to matrix notations., We construct an augmented matrix $X$ by appending a column of 1s in the beginning that will capture $b$.

$X = \begin{pmatrix}1 & x_{11} & x_{12} & \cdots & x_{1M}\\
1 & x_{21} & x_{22} & \cdots & x_{2M}\\
\vdots\\
1 & x_{N1} & x_{N2} & \cdots & x_{NM}\\
\end{pmatrix}_{N\times(M+1)}$

Our iteration vector is $\vec{w}_{1\times(M+1)} = \begin{pmatrix} b \\ w_1 \\ w_2 \\ \vdots \\ w_M \end{pmatrix}_{(M+1)\times 1}$, thus $X\vec{w}$ has dimensions $N \times 1$ which is same as our target vector $\vec{y}$

Let's also define an augmented  vector $\vec{v} = \begin{pmatrix} 0 \\ w_1 \\ w_2 \\ \vdots \\ w_M \end{pmatrix}_{(M+1)\times 1}$

And the update simplifies to:

\begin{align*}
\vec{w}_{(M+1)\times 1}^{(t+1)} &= \vec{w}^{(t+1)}_{(M+1)\times 1} - \frac{\mu}{N} X^T(X\vec{w}^{(t+1)}-\vec{y}) + \frac{\lambda}{M} \vec{v}^{t}
\end{align*}


In [22]:
def split_train_test(X, y, n_train=500):
    nrows = X.shape[0]
    indices = numpy.random.permutation(nrows)
    training_idx, test_idx = indices[:n_train], indices[n_train:]
    X_train, X_test = X[training_idx,:], X[test_idx,:]
    y_train, y_test = y[training_idx], y[test_idx]
    return (X_train, y_train, X_test, y_test)


In [38]:

X_train = []
y_train = []

X_test = []
y_test = []

N_iter = 100
lamda_w = 0.01
T = 500

for i in range(N_iter):
    train_x, train_label, test_x, test_label = split_train_test(df_normalized, output)
    X_train.append(train_x)
    y_train.append(train_label)
    X_test.append(test_x)
    y_test.append(test_label)

def sigmoid(z):
    return 1/(1+np.exp(-z))


def calc_error(z, y_true):
    y_predicted = sigmoid(z)
    y_predicted[y_predicted>0.5] = 1
    y_predicted[y_predicted<=0.5] = 0
    return np.mean(np.abs(y_predicted-y_true))


def logistic_regression(X, y, num_steps=500, learning_rate=1e-6, regularizer_lambda=0.01):
    intercept = np.ones((X.shape[0], 1))
    X = np.hstack((intercept, X))
    weights = np.zeros(X.shape[1])
    regularization_indicator = np.ones(X.shape[1])
    regularization_indicator[0] =  0
    regularization_weights = np.multiply(regularization_indicator, weights)
    training_errors = []
    for step in range(num_steps):
        scores = np.dot(X, weights)
        predictions = sigmoid(scores)
        error = predictions - y
        gradient = np.dot(X.T, error)
        regularization_weights = np.multiply(regularization_indicator, weights)
        regularization = regularizer_lambda * regularization_weights

        weights = weights - (learning_rate * gradient) + regularization
        
        training_error = calc_error(scores, y)
        training_errors.append(training_error)
        print(training_error)
    return weights, training_error


In [40]:
weights, error = logistic_regression(X_train[0], y_train[0], num_steps=500, learning_rate=0.01,
                                     regularizer_lambda=0.01)

0.364
0.364
0.566
0.364
0.526
0.364
0.488
0.364
0.424
0.364
0.354
0.364
0.282
0.364
0.234
0.362
0.174
0.346
0.144
0.296
0.112
0.228
0.098
0.164
0.094
0.132
0.084
0.112
0.082
0.104
0.084
0.086
0.082
0.084
0.08
0.078
0.082
0.078
0.082
0.082
0.082
0.084
0.084
0.082
0.082
0.082
0.082
0.082
0.082
0.08
0.08
0.08
0.082
0.082
0.082
0.082
0.082
0.082
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.082
0.082
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.084
0.086
0.086
0.086
0.086
0.086
0.086
0.084
0.084
0.082
0.08
0.08
0.08
0.08
0.08
0.08
0.08
0.08
0.08
0.08
0.078
0.078
0.078
0.078
0.078
0.078
0.078
0.078
0.078
0.078
0.08
0.08
0.078
0.078
0.078
0.078
0.078
0.08
0.08
0.08
0.08
0.08
0.08
0.08
0.08
0.08
0.08
0.08
0.08
0.08
0.08
0.08
0.08
0.08
0.08
0.08
0.08
0.08
0.082
0.084
0.084
0.084
0.084
0.084
0.084

In [28]:
num_observations = 1000

x1 = np.random.multivariate_normal([0, 0], [[1, .75],[.75, 1]], num_observations)
x2 = np.random.multivariate_normal([1, 4], [[1, .75],[.75, 1]], num_observations)
simulated_separableish_features = np.vstack((x1, x2)).astype(np.float32)
simulated_labels = np.hstack((np.zeros(num_observations),
                              np.ones(num_observations)))
simulated_separableish_features.shape
weights = np.zeros(simulated_separableish_features.shape[1])

In [30]:
np.dot(simulated_separableish_features, weights)

array([0., 0., 0., ..., 0., 0., 0.])

# Problem (c)
Now that we have normalized data, partition it into train/test sets at random 100 times
as discussed earlier. In each trial learn the model by solving (1) with λ = 0.01. To
due this run gradient descent for T = 500 iterations and then use the trained model to
make predictions on the test data and calculate the average error (average number of
miss-classified patients on the test data) for each trial. Report the average over the 100
trials. The value of the step size you use does not matter too much. However, make sure
that the algorithm has converged

# Problem (d)

Perform the experiment with the same step size you used before but now report the
number of iterations it takes to get to an accuracy of 10−6
calculated via the first iteration t when the following inequality holds
k∇f(wt, bt)k2 ≤ 10−6(1 + |f(wt, bt)|)

The number you should report is the average of this number over the 100 trials

# Problem (e)

Perform the experiment of part (d) but now add a momentum term (1) using the heavy
ball method and (2) using Nesterov’s accelerated scheme. In both cases keep the same
step size as part (d) but fine tune the momentum parameter to get the smallest number
of iterations for convergence based on the stopping criteria (2) (again averaged over the
100 trials). Draw the convergence of the three algorithms gradient descent, heavy ball,
Nesterov’s accelerated scheme for one trial. That is, draw the ratio
k∇f(wt, bt)k2`2(1 + |f(wt, bt)|) as a function of the iteration number t = 1, 2, . . . , 500. Which algorithm would you use
and why